In [1]:
from typing import Union, Self
from numbers import Number
from itertools import chain, product

import numpy as np
import sympy as sp
import cvxpy as cp

import scipy
from scipy.sparse import csr_matrix
import scipy.sparse as sparse
from scipy.linalg import qr
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import splu, svds
from sksparse.cholmod import cholesky

from bmn.algebra import MatrixOperator, SingleTraceOperator, MatrixSystem, DoubleTraceOperator
from bmn.linear_algebra import get_null_space_dense, create_sparse_matrix_from_dict, get_row_space_dense, get_null_space_sparse
from bmn.bootstrap import BootstrapSystem
from bmn.solver import (
    minimal_eigval,
    sdp_init, sdp_relax,
    sdp_minimize,
    #minimize,
    get_quadratic_constraint_vector_sparse,
    get_quadratic_constraint_vector_dense,
    compute_L2_norm_of_quadratic_constraints,
)
from bmn.brezin import compute_Brezin_energy, compute_Brezin_energy_Han_conventions
import os
from bmn.debug_utils import disable_debug

import yaml
from bmn.config_utils import (
    run_bootstrap_from_config,
    generate_configs_one_matrix,
    generate_configs_two_matrix,
    generate_configs_three_matrix,
    generate_configs_bfss,
    )

np.set_printoptions(linewidth=120)  # Adjust the number to the desired width

# plot settings
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler
import torch
import torch.optim as optim
from torch.nn import ReLU

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 5.0
plt.rcParams['xtick.minor.size'] = 3.0
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['lines.linewidth'] = 2
plt.rc('font', family='serif',size=16)
matplotlib.rc('text', usetex=True)
matplotlib.rc('legend', fontsize=16)
matplotlib.rcParams['axes.prop_cycle'] = cycler(
    color=['#E24A33', '#348ABD', '#988ED5', '#777777', '#FBC15E', '#8EBA42', '#FFB5B8']
    )
matplotlib.rcParams.update(
    {"axes.grid":True,
    "grid.alpha":0.75,
    "grid.linewidth":0.5}
    )
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
torch.cuda.is_available()

True

In [41]:
from bmn.models import OneMatrix, TwoMatrix, MiniBFSS

model = TwoMatrix(couplings={"g2":0, "g4": 1})
checkpoint_path = "checkpoints/TwoMatrix_L_3_symmetric_energy_fixed_g2_0"

model = MiniBFSS(couplings={"lambda":1})
checkpoint_path = "checkpoints/MiniBFSS_L_3_symmetric"

bootstrap = BootstrapSystem(
    matrix_system=model.matrix_system,
    hamiltonian=model.hamiltonian,
    gauge_generator=model.gauge_generator,
    max_degree_L=3,
    symmetry_generators=model.symmetry_generators,
    verbose=True,
    checkpoint_path=checkpoint_path
    )
bootstrap.load_constraints(checkpoint_path)

# build the Ax = b constraint
Avec = bootstrap.single_trace_to_coefficient_vector(
    SingleTraceOperator(data={(): 1}), return_null_basis=True
)
Avec = torch.from_numpy(Avec).type(torch.float)

# Hamiltonian vector
Hvec = bootstrap.single_trace_to_coefficient_vector(
    bootstrap.hamiltonian, return_null_basis=True
)
Hvec = torch.from_numpy(Hvec).type(torch.float)

Assuming all operators are either Hermitian or anti-Hermitian.
Assuming all operators are either Hermitian or anti-Hermitian.
NOTE Remember to incorporate more general basis changes!
Bootstrap system instantiated for 55987 operators
Attribute: simplify_quadratic = True
Attempting to load from checkpoints, checkpoint_dir=checkpoints/MiniBFSS_L_3_symmetric
  loaded previously computed linear constraints
  loaded previously computed cyclic constraints
  loaded previously computed null space matrix
  loaded previously computed quadratic constraints (numerical)
  loaded previously computed bootstrap table


In [42]:
# build the bootstrap array
bootstrap_array_sparse = bootstrap.bootstrap_table_sparse
bootstrap_array_torch = torch.from_numpy(bootstrap_array_sparse.todense()).type(torch.float)

# build the constraints
quadratic_constraints = bootstrap.quadratic_constraints_numerical
quadratic_constraint_linear = torch.from_numpy(quadratic_constraints["linear"].todense()).type(torch.float)
quadratic_constraint_quadratic = torch.from_numpy(quadratic_constraints["quadratic"].todense()).type(torch.float)
quadratic_constraint_quadratic = quadratic_constraint_quadratic.reshape((len(quadratic_constraint_quadratic), len(Hvec), len(Hvec)))

In [43]:
Avec

tensor([-0.2365,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000])

In [44]:
Hvec

tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00, -2.0671e-01,  0.0000e+00,
        -1.6035e-16,  0.0000e+00,  5.2259e-17, -6.2450e-17,  0.0000e+00,
         2.8189e-16,  0.0000e+00,  3.1963e-17,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         1.2888e-16,  0.0000e+00, -2.5412e-18,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00, -1.4392e-17,  0.0000e+00,
        -1.5331e-17,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  9.7434e-18,  0.0000e+00,
         7.7777e-19,  0.0000e+00, -3.6105e-18,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00, -7.4104e-18,  0.0000e+00,
         9.7060e-18,  0.0000e+00,  0.0000e+00,  6.0090e-19,  2.4961e-18,
         4.9435e-18,  0.0000e+00,  7.5868e-19,  0.0

In [68]:
def energy(param):
    return Hvec @ param


def get_quadratic_constraint_vector(param):
    quadratic_constraints = torch.einsum(
        "Iab, a, b -> I", quadratic_constraint_quadratic, param, param
    ) + torch.einsum("Ia, a -> I", quadratic_constraint_linear, param)
    return torch.square(quadratic_constraints)


def quadratic_loss(param):
    return torch.sum(get_quadratic_constraint_vector(param))


def Axb_loss(param):
    return torch.square(Avec @ param - 1)


def psd_loss(param):
    bootstrap_matrix = (bootstrap_array_torch @ param).reshape(
        (bootstrap.bootstrap_matrix_dim, bootstrap.bootstrap_matrix_dim)
    )  # is this reshaping correct?
    smallest_eigv = torch.linalg.eigvalsh(bootstrap_matrix)[0]
    return ReLU()(-smallest_eigv)


def build_loss(param):
    lambda_psd = 1e2
    lambda_quadratic = 1e2
    lambda_Axb = 1e2
    loss = (
        energy(param)
        + lambda_psd * psd_loss(param)
        + lambda_quadratic * quadratic_loss(param)
        + lambda_Axb * Axb_loss(param)
    )
    return loss

In [75]:
# Training loop
param = 1e0 * torch.randn(bootstrap.param_dim_null)
param.requires_grad = True

optimizer = optim.Adam([param], lr=1e-3)
num_epochs = 100_000

for epoch in range(num_epochs):
    optimizer.zero_grad()  # Clear previous gradients

    loss = build_loss(param)  # Compute the loss
    loss.backward()  # Compute gradients
    optimizer.step()  # Update the parameters

    # Print the loss for monitoring
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

Epoch 100/100000, Loss: 931.7431030273438
Epoch 200/100000, Loss: 764.5570678710938
Epoch 300/100000, Loss: 637.837646484375
Epoch 400/100000, Loss: 538.7415161132812
Epoch 500/100000, Loss: 460.46429443359375
Epoch 600/100000, Loss: 398.54107666015625
Epoch 700/100000, Loss: 349.4683532714844
Epoch 800/100000, Loss: 310.508056640625
Epoch 900/100000, Loss: 279.5779724121094
Epoch 1000/100000, Loss: 255.09182739257812
Epoch 1100/100000, Loss: 235.8155517578125
Epoch 1200/100000, Loss: 220.7638702392578
Epoch 1300/100000, Loss: 209.12326049804688
Epoch 1400/100000, Loss: 200.1971893310547
Epoch 1500/100000, Loss: 193.37098693847656
Epoch 1600/100000, Loss: 188.09930419921875
Epoch 1700/100000, Loss: 183.91207885742188
Epoch 1800/100000, Loss: 180.42864990234375
Epoch 1900/100000, Loss: 177.36634826660156
Epoch 2000/100000, Loss: 174.5359649658203
Epoch 2100/100000, Loss: 171.82290649414062
Epoch 2200/100000, Loss: 169.16470336914062
Epoch 2300/100000, Loss: 166.52972412109375
Epoch 2400

KeyboardInterrupt: 

In [76]:
energy(param)

tensor(1.0875, grad_fn=<DotBackward0>)

In [77]:
psd_loss(param), quadratic_loss(param), Axb_loss(param)

(tensor(2.9313e-07, grad_fn=<ReluBackward0>),
 tensor(6.4238e-05, grad_fn=<SumBackward0>),
 tensor(7.4007e-07, grad_fn=<PowBackward0>))

In [96]:
np.where(np.abs(Hvec.numpy()) > 1e-10)[0]

array([8])

In [101]:
i = 8
param[i]

tensor(-5.2607, grad_fn=<SelectBackward0>)

In [102]:
get_quadratic_constraint_vector(param)

tensor([1.2508e-05, 3.8451e-08, 7.0556e-09, 6.6388e-06, 4.6740e-07, 1.9831e-05,
        2.4503e-05, 2.4392e-07], grad_fn=<PowBackward0>)

In [104]:
84**2

7056

In [103]:
bootstrap_array_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 292273 stored elements and shape (67081, 84)>

In [78]:
param

tensor([-4.2250, -5.8345, -0.6367, -2.3355, -1.6032, -0.0747, -1.7489,  1.2531,
        -5.2607,  0.0148,  0.5487,  1.0272, -0.4563,  4.1943,  0.5406,  0.6841,
        -1.6973,  1.7188,  0.9422, -0.0328, -3.0643, -0.9133,  3.2352,  0.5625,
        -0.3757, -0.0441, -1.3587,  1.1895, -1.1317, -0.7061,  0.4938,  0.5082,
         0.6176,  0.7115, -0.4184, -0.4534,  3.4995, -0.0383, -0.9490, -2.0163,
        -1.6294, -2.7750,  0.9489, -0.4437, -1.7971,  0.2971,  0.5473, -1.6179,
         0.0714,  0.0941,  0.2517,  0.8214,  1.7867, -0.3840, -1.1007,  0.2931,
         0.3678, -0.3431,  1.6438, -1.1183,  0.5184, -0.1923,  0.5915, -0.2848,
         3.0114,  2.7892,  0.4422,  1.9866,  0.0688,  1.1743, -1.6918, -0.2954,
         0.0500, -0.6461, -1.9291, -0.1974,  0.9227,  1.2430, -0.9421,  8.4238,
         4.5283,  2.2347, -0.9071,  0.3629], requires_grad=True)

In [80]:
bootstrap.single_trace_to_coefficient_vector(model.operators_to_track["x_2"], return_null_basis=True)

array([ 0.00000000e+00, -2.86507592e-01,  0.00000000e+00,  6.78384567e-17,  0.00000000e+00, -7.51498006e-18,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e

In [82]:
bootstrap.single_trace_to_coefficient_vector(model.operators_to_track["p_2"], return_null_basis=True)

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -2.75618243e-01,  0.00000000e+00, -2.65141175e-16,  0.00000000e+00,
        1.16278587e-16,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e

In [81]:
bootstrap.single_trace_to_coefficient_vector(model.operators_to_track["x_4"], return_null_basis=True)

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -7.66671051e-04,  0.00000000e+00,  2.37406133e-02,  0.00000000e+00,
        3.06511203e-02,  4.48640762e-01,  0.00000000e+00,  2.72712531e-17,  0.00000000e+00, -9.98898783e-17,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -4.02393971e-17,  0.00000000e+00, -1.29579096e-17,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.11388335e-16,  0.00000000e+00, -3.42451242e-17,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        3.71494411e-17,  0.00000000e+00,  1.54144220e-17,  0.00000000e+00, -3.20824737e-18,  0.00000000e+00,
        0.00000000e

In [84]:
bootstrap_array_torch.shape

torch.Size([67081, 84])

In [92]:
print(list((bootstrap_array_torch @ param).reshape((bootstrap.bootstrap_matrix_dim, bootstrap.bootstrap_matrix_dim)).detach().numpy()))

[array([ 0.9991397 ,  0.5572048 ,  0.        ,  0.        , -0.49956986,  0.        ,  0.        ,  0.        ,
        0.5572048 ,  0.        ,  0.        , -0.49956986,  0.        ,  0.        ,  0.        ,  0.5572048 ,
        0.        ,  0.        , -0.49956986,  0.49956986,  0.        ,  0.        , -0.48331916,  0.        ,
        0.        ,  0.        ,  0.49956986,  0.        ,  0.        , -0.48331916,  0.        ,  0.        ,
        0.        ,  0.49956986,  0.        ,  0.        , -0.48331916,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,


In [ ]:
from bmn.models import TwoMatrix, MiniBFSS

#model = TwoMatrix(couplings={"g2": 0, "g4": 1})
#checkpoint_path = "checkpoints/TwoMatrix_L_3_symmetric_energy_fixed_g2_0"

model = MiniBFSS(couplings={"lambda": 1})
checkpoint_path = "checkpoints/MiniBFSS_L_3_symmetric"

bootstrap = BootstrapSystem(
    matrix_system=model.matrix_system,
    hamiltonian=model.hamiltonian,
    gauge_generator=model.gauge_generator,
    max_degree_L=3,
    symmetry_generators=model.symmetry_generators,
    verbose=True,
    checkpoint_path=checkpoint_path
    )
bootstrap.load_constraints(checkpoint_path)

In [ ]:
st_operator_inhomo_constraints=[
    (SingleTraceOperator(data={(): 1}), 1)
    ]

A = sparse.csr_matrix((0, bootstrap.param_dim_null))
b = np.zeros(0)
for op, value in st_operator_inhomo_constraints:

    A = sparse.vstack(
        [
            A,
            sparse.csr_matrix(
                bootstrap.single_trace_to_coefficient_vector(
                    op, return_null_basis=True
                )
            ),
        ]
    )
    b = np.append(b, value)
linear_inhomogeneous_eq_no_quadratic = (A, b)

In [ ]:
A.todense()

In [ ]:
bootstrap.null_space_matrix.shape

In [ ]:
bootstrap.null_space_matrix[0].todense()

In [ ]:
commutator = model.matrix_system.single_trace_commutator(
    st_operator1=model.hamiltonian,
    st_operator2=SingleTraceOperator(data={("X0", "Pi0"):1, ("X1", "Pi1"):1}),
)

commutator = model.matrix_system.single_trace_commutator(
    st_operator1=SingleTraceOperator(data={("X0", "Pi0"):1, ("X1", "Pi1"):1}),
    st_operator2=model.hamiltonian,
)

commutator_me = SingleTraceOperator(
    data={
    ("Pi0", "Pi0"): -2 / 2,
    ("Pi1", "Pi1"): -2 / 2,
    ("X0", "X0"): -2 * model.couplings["g2"] / 2,
    ("X1", "X1"): -2 * model.couplings["g2"] / 2,
    ("X0", "X1", "X0", "X1"): 4*model.couplings["g4"] / 4,
    ("X1", "X0", "X1", "X0"): 4*model.couplings["g4"] / 4,
    ("X0", "X1", "X1", "X0"): -4*model.couplings["g4"] / 4,
    ("X1", "X0", "X0", "X1"): -4*model.couplings["g4"] / 4,
    }
    )
commutator == -commutator_me

TODO
- consolidate 1, 2, 3, and BFSS models
- consolidate generate_model_X_config 
- move checkpoints, maybe add info on the checkpoints

In [ ]:
generate_configs_bfss(
    config_filename=f"test",
    config_dir="MiniBFSS_L_3_test",
    st_operator_to_minimize="x_2",
    st_operators_evs_to_set={"energy": 1},
    load_from_previously_computed=True,
    impose_symmetries=True,
    reg_decay_rate=0.7880462815669912,
    )

# python bmn/config_utils.py run_bootstrap_from_config -config_filename test -config_dir MiniBFSS_L_3_test

In [ ]:
reg_decay_rate = np.exp(-3 * np.log(10) / (30 - 1))
reg_decay_rate**(30-1), reg_decay_rate

In [ ]:
generate_configs_bfss(
    config_filename=f"test",
    config_dir="MiniBFSS_L_3_test",
    st_operators_evs_to_set={"energy": 1},
    load_from_previously_computed=True,
    impose_symmetries=True,
    tol=1e-2,
    maxiters=50,
    maxiters_cvxpy=10_000,
    init_scale=1e2,
    penalty_reg=1e6,
    reg=1e-4,
    #reg_decay_rate=reg_decay_rate,
    )

# python bmn/config_utils.py run_bootstrap_from_config -config_filename test_2 -config_dir MiniBFSS_L_3_test

In [ ]:
1.9525 / 3

In [ ]:
generate_configs_two_matrix(
    config_filename=f"test_small_mass_hold_energy_fixed",
    config_dir="TwoMatrix_L_3_symmetric",
    st_operator_to_minimize="x_2",
    st_operators_evs_to_set={"energy": 1},
    g2=1e-4,
    g4=1,
    load_from_previously_computed=True,
    impose_symmetries=True,
    )

In [ ]:
generate_configs_three_matrix(
    config_filename=f"test_small_mass",
    config_dir="ThreeMatrix_L_3_symmetric",
    g2=1e-4,
    g4=1,
    load_from_previously_computed=False,
    impose_symmetries=True,
    )

In [ ]:
run_bootstrap_from_config(
    config_filename=f"g4_{str(0.448)}",
    config_dir="TwoMatrix_L_4_symmetric",
    verbose=False,
    )

In [ ]:
for reg in [1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9]:

    generate_configs_bfss(
        config_filename=f"reg_{int(reg)}",
        config_dir="MiniBFSS_L_3_symmetric",
        energy=1,
        st_operator_to_minimize="x_2",
        load_from_previously_computed=True,
        impose_symmetries=True,
        maxiters_cvxpy=5_000,
        maxiters=100,
        radius=1e6,
        reg=reg,
        )

In [ ]:
st_operator_to_minimize = "x_2"

generate_configs_bfss(
    "MiniBFSS_symmetric_L_3",
    energy=1,
    st_operator_to_minimize="x_2",
    load_from_previously_computed=True,
    impose_symmetries=True,
    maxiters_cvxpy=10_000,
    maxiters=100,
    radius=1e8,
    )

generate_configs_bfss(
    "MiniBFSS_L_3",
    energy=1,
    st_operator_to_minimize="x_2",
    load_from_previously_computed=True,
    impose_symmetries=False,
    maxiters_cvxpy=10_000,
    maxiters=100,
    radius=1e8,
    )

#run_bootstrap_from_config("hello", verbose=False)

## BFSS

In [ ]:
L = 2

matrix_system = MatrixSystem(
    operator_basis=["X0", "X1", "X2", "Pi0", "Pi1", "Pi2"],
    commutation_rules_concise={
        ("Pi0", "X0"): 1,  # use Pi' = i P to ensure reality
        ("Pi1", "X1"): 1,
        ("Pi2", "X2"): 1,
    },
    hermitian_dict={
        "Pi0": False,
        "X0": True,
        "Pi1": False,
        "X1": True,
        "Pi2": False,
        "X2": True,
        },
)

# lambda = 1 here
hamiltonian = SingleTraceOperator(
    data={
        ("Pi0", "Pi0"): -0.5,
        ("Pi1", "Pi1"): -0.5,
        ("Pi2", "Pi2"): -0.5,
        # quartic term (XY)
        ("X0", "X1", "X0", "X1"): -1 / 4,
        ("X1", "X0", "X1", "X0"): -1 / 4,
        ("X0", "X1", "X1", "X0"): 1 / 4,
        ("X1", "X0", "X0", "X1"): 1 / 4,
        # quartic term (XZ)
        ("X0", "X2", "X0", "X2"): -1 / 4,
        ("X2", "X0", "X2", "X0"): -1 / 4,
        ("X0", "X2", "X2", "X0"): 1 / 4,
        ("X2", "X0", "X0", "X2"): 1 / 4,
        # quartic term (YZ)
        ("X1", "X2", "X1", "X2"): -1 / 4,
        ("X2", "X1", "X2", "X1"): -1 / 4,
        ("X1", "X2", "X2", "X1"): 1 / 4,
        ("X2", "X1", "X1", "X2"): 1 / 4,
    }
)

# <tr G O > = 0
gauge = MatrixOperator(data={
    ("X0", "Pi0"): 1,
    ("Pi0", "X0"): -1,
    ("X1", "Pi1"): 1,
    ("Pi1", "X1"): -1,
    ("X2", "Pi2"): 1,
    ("Pi2", "X2"): -1,
    (): 3,
    })

bootstrap = BootstrapSystem(
    matrix_system=matrix_system,
    hamiltonian=hamiltonian,
    gauge_generator=gauge,
    max_degree_L=L,
    odd_degree_vanish=True,
    simplify_quadratic=True,
)

bootstrap.build_quadratic_constraints()

quadratic_constraints_numerical = bootstrap.quadratic_constraints_numerical

In [ ]:
quadratic_constraints_numerical

In [ ]:
def newton_pseudoinverse(param, nsteps, quadratic_constraints_numerical):

    for step in range(nsteps):
        val, grad = get_quadratic_constraint_vector_sparse(
            quadratic_constraints=quadratic_constraints_numerical,
            param=param,
            compute_grad=True,
            )
        grad_pinv = np.linalg.pinv(grad)
        #np.allclose(grad @ grad_pinv @ grad, grad)
        #np.allclose(grad.T @ np.linalg.inv(grad @ grad.T), grad_pinv)
        param = param - np.asarray(grad_pinv @ val)[0]
        print(f"Newton's method: step = {step}, val = {np.linalg.norm(val)}")

    return param

In [ ]:
bootstrap.param_dim_null

In [ ]:
167958/3

In [ ]:
param = np.random.normal(size=bootstrap.param_dim_null)
val = get_quadratic_constraint_vector_sparse(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    )

param = newton_pseudoinverse(param=param, nsteps=10, quadratic_constraints_numerical=quadratic_constraints_numerical)
val = get_quadratic_constraint_vector_sparse(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    )

In [ ]:
val.shape, grad.T.shape

In [ ]:
val.shape, grad.shape, grad_pinv.shape

In [ ]:
val2, grad2 = get_quadratic_constraint_vector_sparse(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    compute_grad=True,
    )

In [ ]:
np.sum(np.abs(grad2[0]))

In [ ]:
grad2.shape

In [ ]:
min(np.sum(np.abs(np.asarray(grad2)), axis=1)), max(np.sum(np.abs(np.asarray(grad2)), axis=1))

In [ ]:
min(np.abs(val2)), max(np.abs(val2))

In [ ]:
val1, grad1 = get_quadratic_constraint_vector_dense(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    compute_grad=True,
    )

val2, grad2 = get_quadratic_constraint_vector_sparse(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    compute_grad=True,
    )

In [ ]:
np.allclose(val1, val2), np.allclose(grad1, grad2)

## BMN

In [ ]:
from bmn.models import MiniBMN
from bmn.bootstrap_complex import BootstrapSystemComplex

model = MiniBMN(couplings={"g2": 1, "g4": 1})

bootstrap = BootstrapSystemComplex(
    matrix_system=model.matrix_system,
    hamiltonian=model.hamiltonian,
    gauge_generator=model.gauge_generator,
    max_degree_L=3,
    simplify_quadratic=False,
    symmetry_generators=None,
    fraction_operators_to_retain=0.41,
    )

In [ ]:
bootstrap.build_null_space_matrix()

In [ ]:
bootstrap.build_linear_constraints()

In [ ]:
bootstrap.build_quadratic_constraints()

In [ ]:
bootstrap.quadratic_constraints_numerical["quadratic"][0]

In [ ]:
bootstrap.build_bootstrap_table()

In [ ]:
from bmn.newton_solver import solve_bootstrap

param, optimization_result = solve_bootstrap(
    bootstrap=bootstrap,
    st_operator_to_minimize=bootstrap.hamiltonian,
    )

In [ ]:
x = 1 + 1j
x.real

In [ ]:
x = 1
x.real